In [2]:
import langchain
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

load_dotenv()

google_api_key = os.getenv("GOOGLE_API_KEY")
openai_api_key = os.getenv("OPENAI_API_KEY")

google_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.0-flash", 
    api_key=google_api_key,
    max_tokens=200
)

openai_llm = ChatOpenAI(
    temperature=0, 
    model="gpt-4", 
    api_key=openai_api_key
)


In [41]:
class Person():
    def __init__(self, name, age):
        self.name = name
        self.age = age

p1 = Person("Praveen", 30)
p2 = Person("Trisha", 41)

print(f"{p1.name} is {p1.age} years old.")
print(f"{p2.name} is {p2.age} years old.")

Praveen is 30 years old.
Trisha is 41 years old.


##### String output parser

In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate.from_template(
    "Give me a brief info about this celebrity: {celebrity_name}",
)

chain = prompt | google_llm | StrOutputParser()

res = chain.invoke({"celebrity_name": "Elon Musk"})

print(res)

Hello world


##### Json & Pydantic output parser (with nested object)

In [ ]:
from langchain_core.output_parsers import JsonOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: str = Field(description="Age of the person")
    birth_date: str = Field(description="Date of birth of the person")
    place_of_birth: str = Field(description="Place of birth")


json_parser = JsonOutputParser(pydantic_object=Person)
pydantic_parser = PydanticOutputParser(pydantic_object=Person)

prompt = PromptTemplate.from_template(
"Give me info about this celebrity: {celebrity_name}\n{format_instructions}",
partial_variables={"format_instructions":json_parser.get_format_instructions()}
)


chain = prompt | google_llm | json_parser

chain.invoke({"celebrity_name": "Elon Musk"})

##### CSV list output parser

In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

prompt = PromptTemplate.from_template(
"""Give me name, age and date of birth in dd/mm/yyyy format for this celebrity: {celebrity_name}
\n {format_instructions}""",
partial_variables={"format_instructions":csv_parser.get_format_instructions()}
)

chain = prompt | google_llm | csv_parser

chain.invoke({"celebrity_name": "Elon Musk"})

['Elon Musk', '52', '28/06/1971']

##### Structured Output Parser

In [ ]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

response_schemas = [
    ResponseSchema(name="name", description="name of the person"),
    ResponseSchema(name="age", description="age of the person"),
    ResponseSchema(name="birth_date", description="birth date of the person")
]

csv_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# print(csv_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
"""Give me name, age and date of birth in dd/mm/yyyy format for this celebrity: {celebrity_name}
\n {format_instructions}""",
partial_variables={"format_instructions":csv_parser.get_format_instructions()}
)

# prompt.format(celebrity_name="Elon Musk")

chain = prompt | google_llm | csv_parser

chain.invoke({"celebrity_name": "Elon Musk"})

##### DatetimeOutputParser

In [92]:
from langchain.output_parsers.datetime import DatetimeOutputParser

datetime_parser = DatetimeOutputParser()

# print(datetime_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
"""Give me date of birth: {celebrity_name}. Output only the date time and nothing else.
\n {format_instructions}""",
partial_variables={"format_instructions":datetime_parser.get_format_instructions()}
)

prompt.format(celebrity_name="Elon Musk")

chain = prompt | google_llm | datetime_parser

chain.invoke({"celebrity_name": "Elon Musk"})

datetime.datetime(1971, 6, 28, 0, 0)

##### EnumOutputParser

In [ ]:
from langchain.output_parsers.enum import EnumOutputParser
from enum import Enum

class YesNoMaybe(Enum):
    YES = "yes"
    NO = "no"
    MAYBE = "maybe"

enum_parser = EnumOutputParser(enum=YesNoMaybe)

# print(enum_parser.get_format_instructions())

prompt = PromptTemplate.from_template(
    "Is {celebrity_name} an actor?. \n Format instructions: {format_instructions}. " \
    "Always stick to the format instructions cases in the response",
   partial_variables={"format_instructions":enum_parser.get_format_instructions()}
)

prompt.format(celebrity_name="Elon Musk")

chain = prompt | google_llm | enum_parser

res = chain.invoke({"celebrity_name": "Elon musk"})

print("No") if res is YesNoMaybe.NO else None
print("Yes") if res is YesNoMaybe.YES else None

Yes
